In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import sklearn
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

## Pretty variable display 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## RANDOM STATE
seed=3

In [3]:
df = pd.read_csv('../bballref_data/bpm_clusters12_all_data.csv', index_col=0)
df.head()

,player_id,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,...,cluster_2_bpm,cluster_3_bpm,cluster_4_bpm,cluster_5_bpm,cluster_6_bpm,cluster_7_bpm,cluster_8_bpm,cluster_9_bpm,cluster_10_bpm,cluster_11_bpm
player,,,,,,,,,,,,,,,,,,,,,
Tariq Abdul-Wahad 2000,abdulta01,SG,25,TOT,61,1578,13.6,0.477,0.036,0.299,...,-0.000001,-0.000000,-0.0,-0.0,-0.008897,-0.0,-0.912439,-0.000000,-0.278662,-0.0
Shareef Abdur-Rahim 2000,abdursh01,SF,23,VAN,82,3223,20.2,0.547,0.075,0.431,...,2.199971,0.000024,0.0,0.0,0.000000,0.0,0.000004,0.000000,0.000000,0.0
Ray Allen 2000,allenra02,SG,24,MIL,82,3070,20.6,0.570,0.288,0.282,...,0.000000,3.597134,0.0,0.0,0.000000,0.0,0.000000,0.002866,0.000000,0.0
John Amaechi 2000,amaecjo01,C,29,ORL,80,1684,13.2,0.505,0.009,0.416,...,-0.000005,-0.000000,-0.0,-0.0,-0.000000,-0.0,-2.454038,-0.000000,-0.000212,-0.0
Derek Anderson 2000,anderde01,SG,25,LAC,64,2201,16.9,0.542,0.207,0.359,...,-0.000000,-0.000000,-0.0,-0.0,-0.099991,-0.0,-0.000000,-0.000000,-0.000005,-0.0


In [8]:
sorted(df.columns)

['age',
 'and1s_per_min',
 'ast_pct',
 'ast_per_poss',
 'astd_pts_per_min',
 'avg_dist',
 'blk_pct',
 'blk_per_poss',
 'bpm',
 'cluster_0',
 'cluster_0_bpm',
 'cluster_1',
 'cluster_10',
 'cluster_10_bpm',
 'cluster_11',
 'cluster_11_bpm',
 'cluster_1_bpm',
 'cluster_2',
 'cluster_2_bpm',
 'cluster_3',
 'cluster_3_bpm',
 'cluster_4',
 'cluster_4_bpm',
 'cluster_5',
 'cluster_5_bpm',
 'cluster_6',
 'cluster_6_bpm',
 'cluster_7',
 'cluster_7_bpm',
 'cluster_8',
 'cluster_8_bpm',
 'cluster_9',
 'cluster_9_bpm',
 'dbpm',
 'def_rtg',
 'drawn_shooting_per_min',
 'drb_pct',
 'drb_per_poss',
 'dws',
 'fg2_pct',
 'fg2_per_poss',
 'fg2a_per_poss',
 'fg3_heave',
 'fg3_pct',
 'fg3_per_poss',
 'fg3a_heave',
 'fg3a_per_fga_pct',
 'fg3a_per_poss',
 'fg_dunk_per_min',
 'fg_pct',
 'fg_pct_00_03',
 'fg_pct_03_10',
 'fg_pct_10_16',
 'fg_pct_16_xx',
 'fg_pct_corner3',
 'fg_pct_fg2a',
 'fg_pct_fg3a',
 'fg_per_poss',
 'fga_per_poss',
 'fouls_offensive_per_min',
 'fouls_shooting_per_min',
 'ft_pct',
 'ft_per

### Define some column variables

In [21]:
cluster_cols = [f'cluster_{n}' for n in range(12)]
bpm_cluster_cols = [f'cluster_{n}_bpm' for n in range(12)]

position_cols =  ['pct_c',
                  'pct_pf',
                  'pct_pg',
                  'pct_sf',
                  'pct_sg',]

shooting_location_cols = [
                         'fg_pct_00_03',
                         'fg_pct_03_10',
                         'fg_pct_10_16',
                         'fg_pct_16_xx',
                         'fg_pct_corner3',
                         'fg_pct_fg2a',
                         'fg_pct_fg3a',
                         'fg_per_poss',
                         'fga_per_poss',]

shooting_cols = ['ts_pct',
                 'avg_dist',
                 'fg2_pct',
                 'fg2a_per_poss',
                 'fg3_pct',
                 'fg3a_per_poss',
                 'ft_pct',
                 'fta_per_poss',
                 'fta_per_fga_pct',
                 'pct_fga_fg2a',
                 'pct_fga_fg3a',
                 'pct_fga_00_03',
                 'pct_fga_03_10',
                 'pct_fga_10_16',
                 'pct_fga_16_xx',
                 'pct_fga_dunk',
                 'pct_fga_fg2a',
                 'pct_fga_fg3a',
                 
            
                ]

bio_cols = [  # some of these cols need to switch dtpyes and handle missing
#             'salary',
            'age',
            'height',
            'weight',
#             'nationality'
           ]

value_cols = ['bpm',
              'obpm',
              'dbpm',
              'vorp',
              'per',
              'off_rtg',
              'def_rtg',
              'plus_minus_on',
              'plus_minus_net',
                ]



In [24]:
avg_cluster_players[value_cols].sort_values(by='def_rtg')

,bpm,obpm,dbpm,vorp,per,off_rtg,def_rtg,plus_minus_on,plus_minus_net
label,,,,,,,,,
4,0.728873,-0.157746,0.886620,1.475352,17.377465,113.471831,102.626761,2.127465,2.099296
2,1.982573,1.385892,0.598340,2.431120,19.253942,108.966805,103.232365,1.952697,3.263071
7,-1.255345,-1.506294,0.252947,0.422178,14.620080,107.714286,104.118881,-0.751948,-0.300899
0,-1.322936,-1.429969,0.111315,0.352294,14.567278,105.235474,104.180428,-1.172477,-0.343731
9,4.315337,3.847853,0.469018,4.272086,21.863497,111.868098,104.748466,3.576074,5.765337
8,-0.428423,-0.678571,0.252381,0.696875,14.327232,107.940476,106.388393,-0.509673,-0.215327
10,-2.338958,-2.211166,-0.125062,-0.078660,11.541439,99.578164,106.962779,-2.502481,-2.019603
3,1.971181,2.133228,-0.161575,2.597480,17.849764,109.697638,107.434646,2.098110,3.211969
1,-0.527465,-0.394836,-0.134507,0.743662,12.612559,107.654930,108.151408,0.102465,0.269601


## Generate mean player for each cluster

In [4]:
cluster_groupby = df.groupby('label')

In [6]:
avg_cluster_players = cluster_groupby.mean()

In [7]:
avg_cluster_players

,age,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,...,cluster_2_bpm,cluster_3_bpm,cluster_4_bpm,cluster_5_bpm,cluster_6_bpm,cluster_7_bpm,cluster_8_bpm,cluster_9_bpm,cluster_10_bpm,cluster_11_bpm
label,,,,,,,,,,,,,,,,,,,,,
0,27.241590,68.155963,1545.180428,14.567278,0.523263,0.011171,0.332398,9.484709,18.949541,14.204893,...,-0.002178,0.000000,-0.001044,0.000000e+00,0.000000e+00,-0.001458,-0.001871,-2.136697e-06,-1.832537e-02,0.000000
1,26.876761,72.576291,1908.225352,12.612559,0.542762,0.435413,0.209566,3.324883,12.446244,7.876995,...,-0.000245,0.001802,0.000000,-9.069528e-04,-1.278361e-02,0.000000,-0.010532,6.715048e-04,-1.767163e-03,-0.030341
2,26.763485,71.842324,2375.514523,19.253942,0.544568,0.042303,0.353373,8.230290,20.906224,14.595851,...,1.973992,0.000161,-0.000116,0.000000e+00,-6.182573e-08,0.000009,0.007928,4.667167e-03,-6.588595e-03,0.000000
3,26.738583,73.951181,2592.014173,17.849764,0.547647,0.285017,0.292098,2.779685,11.271969,7.037480,...,-0.000119,1.930045,0.000000,4.755396e-03,7.498667e-03,0.000000,0.000503,2.869271e-02,-2.780446e-04,0.000000
4,26.711268,69.908451,1771.556338,17.377465,0.573789,0.011669,0.555232,11.723944,23.334507,17.551408,...,0.002094,0.000000,0.728127,0.000000e+00,0.000000e+00,-0.003304,0.000245,4.049296e-07,-3.951408e-06,0.000000
5,28.124172,66.546358,1594.584437,13.359106,0.519998,0.356444,0.212382,1.839238,9.095199,5.461424,...,0.000000,0.003103,0.000000,-7.282069e-01,-1.216468e-02,0.000000,0.000012,2.091060e-07,-7.866895e-03,-0.013713
6,26.382677,65.088189,1832.307087,14.988189,0.532027,0.321969,0.280200,2.723150,11.365512,7.029921,...,0.000448,0.001257,0.000000,-6.560810e-03,8.016247e-02,0.000000,-0.000784,3.137871e-03,-5.408952e-03,-0.025134
7,26.800200,66.779221,1489.058941,14.620080,0.543252,0.002860,0.395970,10.572727,20.313287,15.439660,...,0.000055,0.000000,0.000091,0.000000e+00,0.000000e+00,-1.254523,0.000000,4.675325e-08,-3.686314e-08,0.000000
8,26.061012,66.561012,1544.331845,14.327232,0.542217,0.219528,0.284482,6.929167,16.962202,11.938095,...,0.001152,0.000083,-0.000104,8.265476e-06,1.447230e-03,0.000000,-0.375107,8.436283e-04,-2.135789e-02,-0.017916
